In [86]:
import uproot, vector, awkward as ak

filename="user.tapark.41935082._000001.tree.root"
file = uproot.open(filename)
tree = file["JetConstituentTree"]
# tree.show()
# to read TBranch ; NOTE: use entry_start, entry_stop to read only specific amounts of entry for optimization
tjets = tree.arrays(['truthJet_pt', 'truthJet_eta', 'truthJet_phi', 'truthJet_m'], library="np")
# add filtering on truth jets: cut="truthJet_m >= 20000"
reco_jets = tree.arrays(['jet_pt', 'jet_eta', 'jet_phi', 'jet_m'], library="np")


In [39]:
# filter M = 20 GeV truth jets
m20gev_tjets = [ event for event in truth_jets['truthJet_m'] if any(y>=20000 for y in event) ]
m20gev_tjets

[]

In [ ]:
import numpy as np
import momentum4D_cython


def main():

    # FILTER INPUT DATA HERE
    truth_jets_pt = [ event for event in tjets["truthJet_pt"] if len(event)!=0 ]
    truth_jets_phi = [ event for event in tjets["truthJet_phi"] if len(event)!=0 ]
    truth_jets_eta = [ event for event in tjets["truthJet_eta"] if len(event)!=0 ]
    truth_jets_m = [ event for event in tjets["truthJet_m"] if len(event)!=0 ]

    # PASS DATA TO CYTHON SCRIPT
    momentum_objects = momentum4D_cython.compute_momentum4d(truth_jets_pt, truth_jets_phi, truth_jets_eta, truth_jets_m)
    return momentum_objects[:10]

# main()
# TRY AWKWARD ARRAYS ZIP AND PASS THAT THRU CYTHON

TypeError: Argument 'pt_values' has incorrect type (expected numpy.ndarray, got list)